# Agenda, day 4 — text and dates
 
1. Q&A
2. Textual data and Pandas
3. Cleaning dirty textual data
4. Statistics about text
5. Useful string methods
6. Time and date information
    - `datetime` 
    - `timedelta`
7. Calculating time deltas
8. Time series (i.e., where we have time data as our index)
9. Resampling